In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../"]).decode("utf8"))

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2
from tqdm import tqdm
from keras import optimizers

from sklearn.cross_validation import KFold
from sklearn.metrics import fbeta_score
import time

x_train = []
x_test = []
y_train = []

df_train = pd.read_csv('train_v2.csv')
df_test = pd.read_csv('sample_submission_v2.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

count = 0
for f, tags in tqdm(df_train.values, miniters=1000):
    if count < 2000:
        img = cv2.imread('./train-jpg/{}.jpg'.format(f))
        targets = np.zeros(17)
        for t in tags.split(' '):
            targets[label_map[t]] = 1 
        x_train.append(cv2.resize(img, (64, 64)))
        y_train.append(targets)
        count += 1

count = 0
for f, tags in tqdm(df_test.values, miniters=1000):
    if count < 5000:
        img = cv2.imread('./test-jpg/{}.jpg'.format(f))
        x_test.append(cv2.resize(img, (64, 64)))
        count += 1
    


GEC_Lab_IntroLevel
GEC_Lab_Level2
Kaggle-planet-test-tif.torrent
Kaggle-planet-train-tif.torrent
Report 1 for ML Program on image Liang Zhang.docx
Report 2 for ML Program on image Liang Zhang.docx
UCB-ImageProject
sample_submission_v2.csv
sample_submission_v2.csv.zip
test-jpg-additional.tar
test-jpg-additional.tar.7z
test-jpg.tar
test-jpg.tar.7z
test-tif-v2.tar.7z
test_v2_file_mapping.csv.zip
train-jpg.tar
train-jpg.tar.7z
train-tif-v2.tar.7z
train_v2.csv.zip
~$port 1 for ML Program on image Liang Zhang.docx
~$port 2 for ML Program on image Liang Zhang.docx



100%|██████████| 61191/61191 [00:22<00:00, 2674.34it/s]


In [ ]:
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)/255.
x_test  = np.array(x_test, np.float32)/255.

print(x_train.shape)
print(y_train.shape)

#nfolds = 5
nfolds = 4

num_fold = 0
sum_score = 0

yfull_test = []
yfull_train =[]

kf = KFold(len(y_train), n_folds=nfolds, shuffle=True, random_state=1)

for train_index, test_index in kf:
        start_time_model_fitting = time.time()
        
        X_train = x_train[train_index]
        Y_train = y_train[train_index]
        X_valid = x_train[test_index]
        Y_valid = y_train[test_index]

        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))
        
        kfold_weights_path = os.path.join('', 'weights_kfold_' + str(num_fold) + '.h5')
        
        model = Sequential()
        model.add(BatchNormalization(input_shape=(64, 64,3)))
        model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(32, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(256, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(17, activation='sigmoid'))

        epochs_arr = [20, 5, 5]
        learn_rates = [0.001, 0.0001, 0.00001]

        for learn_rate, epochs in zip(learn_rates, epochs_arr):
            opt  = optimizers.Adam(lr=learn_rate)
            model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                          optimizer=opt,
                          metrics=['accuracy'])
            callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0),
            ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)]

            model.fit(x = X_train, y= Y_train, validation_data=(X_valid, Y_valid),
                  batch_size=128,verbose=2, epochs=epochs,callbacks=callbacks,shuffle=True)
        
        if os.path.isfile(kfold_weights_path):
            model.load_weights(kfold_weights_path)
        
        p_valid = model.predict(X_valid, batch_size = 128, verbose=2)
        print("final fbeta_score: ", fbeta_score(Y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

        p_train = model.predict(x_train, batch_size =128, verbose=2)
        yfull_train.append(p_train)
        
        p_test = model.predict(x_test, batch_size = 128, verbose=2)
        yfull_test.append(p_test)


(2000, 64, 64, 3)
(2000, 17)
Start KFold number 1 from 4
Split train:  1500 1500
Split valid:  500 500
Train on 1500 samples, validate on 500 samples
Epoch 1/20
73s - loss: 0.8354 - acc: 0.5099 - val_loss: 5.3216 - val_acc: 0.3094
Epoch 2/20
71s - loss: 0.8749 - acc: 0.5277 - val_loss: 5.3790 - val_acc: 0.2739
Epoch 3/20
72s - loss: 0.7780 - acc: 0.5664 - val_loss: 4.4221 - val_acc: 0.1555
Epoch 4/20
497s - loss: 0.7158 - acc: 0.6084 - val_loss: 2.5140 - val_acc: 0.1186
Epoch 5/20
75s - loss: 0.6225 - acc: 0.6758 - val_loss: 1.5628 - val_acc: 0.2028
Epoch 6/20
73s - loss: 0.5290 - acc: 0.7522 - val_loss: 1.3264 - val_acc: 0.2118
Epoch 7/20
72s - loss: 0.4519 - acc: 0.8140 - val_loss: 0.9337 - val_acc: 0.3209
Epoch 8/20
72s - loss: 0.3957 - acc: 0.8519 - val_loss: 0.5584 - val_acc: 0.7607
Epoch 9/20
73s - loss: 0.3453 - acc: 0.8799 - val_loss: 0.6258 - val_acc: 0.6000
Epoch 10/20
75s - loss: 0.3239 - acc: 0.8897 - val_loss: 0.6259 - val_acc: 0.6946
Epoch 11/20
72s - loss: 0.3108 - acc: 

91s - loss: 0.5557 - acc: 0.9051 - val_loss: 0.4992 - val_acc: 0.9085
Epoch 4/20
94s - loss: 0.4503 - acc: 0.9051 - val_loss: 0.3878 - val_acc: 0.9085
Epoch 5/20
96s - loss: 0.3509 - acc: 0.9051 - val_loss: 0.3100 - val_acc: 0.9085
Epoch 6/20
115s - loss: 0.2958 - acc: 0.9051 - val_loss: 0.2763 - val_acc: 0.9085
Epoch 7/20
103s - loss: 0.2742 - acc: 0.9051 - val_loss: 0.2650 - val_acc: 0.9085
Epoch 8/20
103s - loss: 0.2663 - acc: 0.9051 - val_loss: 0.2608 - val_acc: 0.9085
Epoch 9/20
104s - loss: 0.2624 - acc: 0.9051 - val_loss: 0.2590 - val_acc: 0.9085
Epoch 10/20
93s - loss: 0.2604 - acc: 0.9051 - val_loss: 0.2579 - val_acc: 0.9085
Epoch 11/20
94s - loss: 0.2593 - acc: 0.9051 - val_loss: 0.2571 - val_acc: 0.9085
Epoch 12/20
119s - loss: 0.2588 - acc: 0.9051 - val_loss: 0.2565 - val_acc: 0.9085
Epoch 13/20
122s - loss: 0.2580 - acc: 0.9051 - val_loss: 0.2560 - val_acc: 0.9085
Epoch 14/20
107s - loss: 0.2574 - acc: 0.9051 - val_loss: 0.2557 - val_acc: 0.9085
Epoch 15/20
104s - loss: 0.